In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)


In [8]:
X_train.numpy()[0:5]

array([[-0.36180827, -0.26521012, -0.31715703, -0.4671384 ,  1.8038261 ,
         1.1817418 , -0.5168924 ,  0.10653677, -0.3900515 ,  1.3914014 ,
         0.1437097 , -0.12075458,  0.16013007, -0.13255125, -0.58631635,
        -0.12476339, -0.57865274,  0.10906824, -0.28187352, -0.1889237 ,
        -0.25710148, -0.24033175, -0.2441722 , -0.36688232,  0.5448709 ,
         0.24812607, -0.7108862 , -0.07967529, -0.5279873 ,  0.2506337 ],
       [-0.8632675 ,  0.71560603, -0.85646015, -0.7966804 , -0.05863438,
        -0.42846033, -0.5170174 , -0.6814299 ,  0.7947534 ,  0.38824034,
        -0.45446828,  0.40088484, -0.43568602, -0.5215511 , -1.1630561 ,
         0.27242032,  0.06745547, -0.2391592 ,  1.1130354 ,  0.35017294,
        -0.8894322 ,  0.38469607, -0.8880358 , -0.7896815 , -1.042863  ,
        -0.48243034, -0.5630865 , -0.7698024 ,  0.44312727, -0.20987332],
       [-0.4334453 ,  0.32513896, -0.41286668, -0.5036265 ,  0.20285109,
         0.31694695,  0.21135233,  0.29227546, -0

In [3]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Converting to tensors

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# Converting to column vectors
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [4]:
# Linear model f = wx + b , sigmoid at the end

class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1) # n_input_features is the number of features in the dataset and we have 1 output

    def forward(self, x):
        linearResult = self.linear(x)
        y_pred = torch.sigmoid(linearResult)
        return y_pred


In [5]:

model = Model(n_features)

# 2) Loss and optimizer
num_epochs = 100
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) Training loop
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')


with torch.no_grad():
    
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')


c:\Users\itsab\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


epoch: 10, loss = 0.5217
epoch: 20, loss = 0.4412
epoch: 30, loss = 0.3875
epoch: 40, loss = 0.3490
epoch: 50, loss = 0.3198
epoch: 60, loss = 0.2968
epoch: 70, loss = 0.2781
epoch: 80, loss = 0.2626
epoch: 90, loss = 0.2494
epoch: 100, loss = 0.2380
accuracy: 0.9035
